In [11]:
# imports
import os
import openai
import pickle
import getpass
from openai import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document

In [2]:
# Configuração da chave da API OpenAI
os.environ['OPENAI_API_KEY'] = "sk-proj-MtKau4PXp2gUE3CeN8lnT3BlbkFJTotqyxDKzqC3GJ2xF7Ki"

# Solicita a chave da API
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: sk-proj-MtKau4PXp2gUE3CeN8lnT3BlbkFJTotqyxDKzqC3GJ2xF7Ki")

In [3]:
# Define o caminho relativo para a pasta 'splits' dentro do projeto
current_directory = os.getcwd()
splits_directory = os.path.join(current_directory, 'splits')
splits_filepath = os.path.join(splits_directory, 'splits.pkl')

In [4]:
# Função para carregar os splits
def load_splits(filepath):
    with open(filepath, 'rb') as file:
        splits = pickle.load(file)
    return splits

# Carregar os splits usando o caminho relativo
splits = load_splits(splits_filepath)

In [6]:
# Definindo os embeddings antes de carregar o FAISS
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

c:\Users\gust_\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
# Carregando o índice FAISS localmente
faiss_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [8]:
# Criar o retriever a partir do faiss_db
retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [12]:
client = OpenAI()

def generate_response(query):
    # Recuperar as 5 combinações mais relevantes
    relevant_docs = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in relevant_docs])

    prompt = f"""
    Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nos documentos fornecidos. Responda sempre em português.

    Contexto: {context}

    Pergunta: {query}
    Resposta:
    """
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nas instruções e no contexto fornecido. Responda sempre em português."},
            {"role": "user", "content": f"Contexto: {context} Pergunta: {query}"}
        ],
        max_tokens=150,
        temperature=0.5
    )

    #print(completion.choices[0].message)
    return completion.choices[0].message.content


In [13]:
# Exemplo de uso
query = "Quem é o representante dos Recursos HUmanos do Shopping Cidadão?"
resposta = generate_response(query)
print(resposta)

Os representantes dos Recursos Humanos do Shopping Cidadão são Carlos Fernandez Rodriguez, Priscila Fonseca e Luciano Catarino.


In [95]:
print(type(retriever))


<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [96]:
print(dir(retriever))


['Config', 'InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__ror__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '__weakref__', '_abatch_with_confi

In [93]:
retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [100]:
query = "Quem é o representante dos Recursos Humanos nos documentos fornecidos?"
relevant_docs = retriever.invoke(query)
print(relevant_docs)


[Document(metadata={'filename': 'introducao_a_gestao_de_RH.pdf'}, page_content='CAPA\nBárbara Bela Editora Gráfica\nDIAGRAMAÇÃO E ARTE FINAL\nBárbara Bela Editora Gráfica\nTIRAGEM\n200 exemplares\nÉ permitida a reprodução parcial ou total desde que citada a fonte.\nBrasil. Ministério do Planejamento, Orçamento e Gestão. Secretaria de Gestão. Comissão\nEuropéia. Introdução à Gestão de Recursos Humanos: texto de referência para a área\ntemática de gestão de pessoas / Ministério do Planejamento, Orçamento e Gestão.Secretaria de Recursos Humanos; Björn Bengtson e Göran Järvstrand; tradução deLuiz S. Macedo de Oliveira. — Brasília: MP ,  2006.\n52 p.\np.1. Administração de pessoal.  I. Projeto euroBrasil 2000 - Comissão Européia.\nCDU (658.3)Apresentação .................................................................................................................. 0 7\nA moderna Administração de Recursos Humanos ........................................................... 09\nPlanejamento

In [89]:
from openai import OpenAI
client = OpenAI()

def chat(query):
    # Recuperar as 5 combinações mais relevantes
    relevant_docs = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in relevant_docs])

    prompt = f"""
    Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nas instruções e no contexto fornecido. Responda sempre em português.

    Contexto: {context}

    Pergunta: {query}
    Resposta:
    """
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um assistente de perguntas e respostas. Seu objetivo é responder perguntas com a maior precisão possível com base nas instruções e no contexto fornecido. Responda sempre em português."},
            {"role": "user", "content": f"Contexto: {context} Pergunta: {query}"}
        ],
        max_tokens=150,
        temperature=0.5
    )

    print(completion.choices[0].message)
    #return chat_response.choices[0].message['content'].strip()

In [90]:
query = "Quem é o representante dos Recursos HUmanos nos documentos fornecidos?"
resposta = chat(query)
print(resposta)

AttributeError: module 'openai' has no attribute 'error'